In [ ]:
In [14]:
#Imports and constants 
################################################################

import csv
import graphviz
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
from sklearn import tree
from sklearn import svm

from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_selection import RFE,SelectKBest,chi2
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, label_binarize, Imputer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

from IPython.display import display, HTML


#set numpy random seed for reproducible results
np.random.seed(1234)
#to view the whole output in the jupyter console (not only an extract)
np.set_printoptions(threshold=np.nan)

#Data parameter
#################################################################
#File paths
TEST_PATH ='dataset/aps_failure_test_set.csv'
TRAIN_PATH ='dataset/aps_failure_training_set.csv'

#Columns of the readed dataframe
COLUMNS = ['aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000', 'ag_001', 'ag_002', 'ag_003', 'ag_004', 'ag_005', 'ag_006', 'ag_007', 'ag_008', 'ag_009', 'ah_000', 'ai_000', 'aj_000', 'ak_000', 'al_000', 'am_0', 'an_000', 'ao_000', 'ap_000', 'aq_000', 'ar_000', 'as_000', 'at_000', 'au_000', 'av_000', 'ax_000', 'ay_000', 'ay_001', 'ay_002', 'ay_003', 'ay_004', 'ay_005', 'ay_006', 'ay_007', 'ay_008', 'ay_009', 'az_000', 'az_001', 'az_002', 'az_003', 'az_004', 'az_005', 'az_006', 'az_007', 'az_008', 'az_009', 'ba_000', 'ba_001', 'ba_002', 'ba_003', 'ba_004', 'ba_005', 'ba_006', 'ba_007', 'ba_008', 'ba_009', 'bb_000', 'bc_000', 'bd_000', 'be_000', 'bf_000', 'bg_000', 'bh_000', 'bi_000', 'bj_000', 'bk_000', 'bl_000', 'bm_000', 'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000', 'bs_000', 'bt_000', 'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000', 'ca_000', 'cb_000', 'cc_000', 'cd_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000', 'ci_000', 'cj_000', 'ck_000', 'cl_000', 'cm_000', 'cn_000', 'cn_001', 'cn_002', 'cn_003', 'cn_004', 'cn_005', 'cn_006', 'cn_007', 'cn_008', 'cn_009', 'co_000', 'cp_000', 'cq_000', 'cr_000', 'cs_000', 'cs_001', 'cs_002', 'cs_003', 'cs_004', 'cs_005', 'cs_006', 'cs_007', 'cs_008', 'cs_009', 'ct_000', 'cu_000', 'cv_000', 'cx_000', 'cy_000', 'cz_000', 'da_000', 'db_000', 'dc_000', 'dd_000', 'de_000', 'df_000', 'dg_000', 'dh_000', 'di_000', 'dj_000', 'dk_000', 'dl_000', 'dm_000', 'dn_000', 'do_000', 'dp_000', 'dq_000', 'dr_000', 'ds_000', 'dt_000', 'du_000', 'dv_000', 'dx_000', 'dy_000', 'dz_000', 'ea_000', 'eb_000', 'ec_00', 'ed_000', 'ee_000', 'ee_001', 'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008', 'ee_009', 'ef_000', 'eg_000']
#Columns of the the final dataframe with processed histograms
HISTOGRAMMERGEDCOLUMNS = ['aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000',
                           'ah_000', 'ai_000',
                          'aj_000', 'ak_000', 'al_000', 'am_0', 'an_000', 'ao_000', 'ap_000',
                          'aq_000', 'ar_000', 'as_000', 'at_000', 'au_000', 'av_000', 'ax_000',
                             'bb_000', 'bc_000', 'bd_000', 'be_000', 'bf_000', 
                          'bg_000', 'bh_000', 'bi_000', 'bj_000', 'bk_000', 'bl_000', 'bm_000', 
                          'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000', 'bs_000', 'bt_000', 
                          'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000', 'ca_000', 'cb_000', 
                          'cc_000', 'cd_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000', 'ci_000', 
                          'cj_000', 'ck_000', 'cl_000', 'cm_000', 
                          'co_000', 'cp_000', 'cq_000', 'cr_000',  
                          'ct_000', 'cu_000', 'cv_000', 'cx_000', 'cy_000', 'cz_000', 'da_000', 
                          'db_000', 'dc_000', 'dd_000', 'de_000', 'df_000', 'dg_000', 'dh_000', 
                          'di_000', 'dj_000', 'dk_000', 'dl_000', 'dm_000', 'dn_000', 'do_000', 
                          'dp_000', 'dq_000', 'dr_000', 'ds_000', 'dt_000', 'du_000', 'dv_000', 
                          'dx_000', 'dy_000', 'dz_000', 'ea_000', 'eb_000', 'ec_00', 'ed_000', 
                          'ef_000', 'eg_000','ag_median','ay_median', 'az_median', 'ba_median','cn_median','cs_median','ee_median']

#possible Class Labels in the dataset
CLASS_LABELS = ['neg', 'pos']
#Parameter to skip the first [SKIPROWS] lines in the csv files before reading into the dataframe
SKIPROWS = 20

#Histogram columns
AG = np.array(['ag_000', 'ag_001', 'ag_002', 'ag_003', 'ag_004', 'ag_005', 'ag_006', 'ag_007', 'ag_008', 'ag_009'])
AY = np.array(['ay_000', 'ay_001', 'ay_002', 'ay_003', 'ay_004', 'ay_005', 'ay_006', 'ay_007', 'ay_008', 'ay_009'])
AZ = np.array(['az_000', 'az_001', 'az_002', 'az_003', 'az_004', 'az_005', 'az_006', 'az_007', 'az_008', 'az_009'])
BA = np.array(['ba_000', 'ba_001', 'ba_002', 'ba_003', 'ba_004', 'ba_005', 'ba_006', 'ba_007', 'ba_008', 'ba_009'])
CN = np.array(['cn_000', 'cn_001', 'cn_002', 'cn_003', 'cn_004', 'cn_005', 'cn_006', 'cn_007', 'cn_008', 'cn_009'])
CS = np.array(['cs_000', 'cs_001', 'cs_002', 'cs_003', 'cs_004', 'cs_005', 'cs_006', 'cs_007', 'cs_008', 'cs_009'])
EE = np.array(['ee_000', 'ee_001', 'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008', 'ee_009'])

#NAN Data preprocessing parameter
NaNSTRATEGY = ['mean', 'median', 'most_frequent']
In [3]:
#All Methods dealing with the data preprocessing 
##################################################################################

#Reads the csv file from the given path as a pandas dataframe
#returns the dataframe
def loadDatasetWithPandas(path, skiprowsNum):
    # Reading the raw data from csv file
    rawData = pd.read_csv(path, skiprows=skiprowsNum)
    # replacing the string indicating missing values with the numpy value for missing values
    NaNProcessedData = rawData.replace({'na': np.nan}, regex=True)
    return NaNProcessedData

#creates a new dataframe from the given datafame where the different histogram bins are processed into a new feature.
#returned dataframe is structured according to 'HISTOGRAMMERGEDCOLUMNS' variable
def histogramProcessing(dataf):
    data_set=dataf
    ag_ = data_set[AG]
    ay_ = data_set[AY]
    az_ = data_set[AZ]
    ba_ = data_set[BA]
    cn_ = data_set[CN]
    cs_ = data_set[CS]
    ee_ = data_set[EE]
    
    # create new dataframe for each of the above with the mean\n",
    ag_mean = ag_.mean(axis=1, skipna=True)
    ay_mean = ay_.mean(axis=1, skipna=True)
    az_mean = az_.mean(axis=1, skipna=True)
    ba_mean = ba_.mean(axis=1, skipna=True)
    cn_mean = cn_.mean(axis=1, skipna=True)
    cs_mean = cs_.mean(axis=1, skipna=True)
    ee_mean = ee_.mean(axis=1, skipna=True)

    remaining_columns = np.setdiff1d(np.setdiff1d(np.setdiff1d(np.setdiff1d(np.setdiff1d(np.setdiff1d(np.setdiff1d(np.array(COLUMNS), AG), AY), AZ), BA), CN), CS), EE)
    remaining_columns= np.concatenate([['class'],remaining_columns])
    remaining_data = data_set[remaining_columns]
    return pd.concat(
        [
            remaining_data,
            ag_mean.rename('ag_median'),
            ay_mean.rename('ay_median'),
            az_mean.rename('az_median'),
            ba_mean.rename('ba_median'),
            cn_mean.rename('cn_median'),
            cs_mean.rename('cs_median'),
            ee_mean.rename('ee_median')
        ],
        axis=1
    )

#processes the NaN values in data frame according to the provided strategy (mean, median or most_frequent) usinjg scikits Imputer.
# returns a dataframe without any NaN values
def processNaNInDataset(data, strategy):
    values = data[list(COLUMNS)].values
    imp = Imputer(missing_values='NaN', strategy=strategy, axis=0)
    imp = imp.fit(values)
    cleanedValues = imp.transform(values)
    return cleanedValues

#loads the csv and uses the 3 methods above to process file (loading, Nan processing)
#returns the loaded testdatframe and trainingdataframe
def loadDataset(strategy):
    #Load Train Dataset
    trainData = loadDatasetWithPandas(TRAIN_PATH, SKIPROWS)
    #Load Test Dataset
    testData = loadDatasetWithPandas(TEST_PATH, SKIPROWS)
    #if no strategy for nan processing is provided just return the laoded dataframes
    if(strategy == None):
        return trainData, testData

    #remove class column from datset to allow conversion to float datatype
    removeClassTrain = trainData.iloc[:,1:171]
    removeClassTest = testData.iloc[:,1:171]
    # change datatype of the dataframe to allow computing of the mean of a column (otherwise an overflow will happen)
    removeClassTrain = removeClassTrain.astype('float64')
    processedTrain = processNaNInDataset(removeClassTrain,strategy)

    removeClassTest = removeClassTest.astype('float64')
    processedTest = processNaNInDataset(removeClassTest,strategy)
   

    finalTrainFrame = pd.concat(
            [
                pd.DataFrame(trainData.iloc[:,0]),
                pd.DataFrame(processedTrain, columns=COLUMNS)
            ],
            axis=1
        )

    finalTestFrame = pd.concat(
            [
                pd.DataFrame(testData.iloc[:,0]),
                pd.DataFrame(processedTest, columns=COLUMNS)
            ],
            axis=1
        )
    return finalTrainFrame, finalTestFrame
In [31]:
#This Method reduces features based the Correlation between the features.
def CorrelationFeatureSelection(DataFrame,X,Y,flag):
    #####Feature Selection with Co-relation####
    # Create correlation matrix
    corr_matrix = X.corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    # Find index of feature columns with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    # Drop features
    # newDataFrame = X.drop(X.columns[to_drop], axis=1)
    print("Features to Drop ")
    print(to_drop)

    # Creating the DataFrame with Selected Features.
    if flag == "Train":
        Readydf = DataFrame[DataFrame.columns.difference(to_drop)]


    elif flag == "Test":
        Readydf = DataFrame[DataFrame.columns.difference(to_drop)]

    return Readydf


#This Method reduces features selecting the K best features using Chi Square.
#X = feature set, Y = class lable values, FtreNo = No of features to be selected.
def SelectKbest(DataFrame,X,Y,FtreNo,flag):

    #####Feature Extraction with Univariate Statistical Tests####

    # feature extraction
    selector = SelectKBest(score_func=chi2, k=FtreNo)
    fit = selector.fit(X, Y)
    # summarize scores
    np.set_printoptions(precision=3)

    #print("Fit Score")
    #print(fit.scores_)
    features = fit.transform(X)

    # get selected feature names
    Selected_feature_names = X.columns[selector.get_support()]

    #Creating the DataFrame with Selected Features.
    if flag == "Train":
        Readydf = pd.DataFrame(DataFrame, columns=Selected_feature_names)

    elif flag == "Test":
        Readydf = pd.DataFrame(DataFrame, columns=Selected_feature_names)

    return Readydf

######Feature elimination by low varience####
def lowVarfilterfeatures(DataFrame):
    print("The Std Dev are: ")
    pd.options.display.float_format = '{:.5f}'.format
    #print(preprocessedData)
    print(DataFrame.std())
    #print(DataFrame.std() > 2.90)
    #print(DataFrame.drop(DataFrame.var()[DataFrame.var() > 2.90].index.values, axis=1))


def filterfeatures(DataFrame, FtreSelMthd, FtreNo, flag):

    #array = ImputedDataFrame.values
    #Ftr = array[:, 0:107]
    #ClsLbl = array[:, 107]


    # Seperating the features from the class lable
    X = DataFrame[DataFrame.columns.difference(['class'])]
    Y = pd.DataFrame(DataFrame, columns=['class'])

    #Based on the feature selection technique the respective method will be called.
    if FtreSelMthd == "Corelation" :
        FeatureSelected_Df = CorrelationFeatureSelection(DataFrame,X,Y,flag)
    elif FtreSelMthd == "SelectKBest":
        FeatureSelected_Df =SelectKbest(DataFrame,X,Y,FtreNo,flag)
        
    FeatureSelected_Df = pd.concat([FeatureSelected_Df,DataFrame[['class']]], axis = 1)

    return FeatureSelected_Df
In [5]:
#All methods dealing with evaluation
#################################################################################

#computes/plots scikit classification report, confusion matrix and calculates the overall cost of the result using the formula provided in the dataset
def evaluate(trueLabels, predictions):
    classificationRep = classification_report(trueLabels, predictions)
    print(classificationRep)
    
    confusionMatrix = confusion_matrix(trueLabels,predictions) 
    #print(confusionMatrix)
    #np.set_printoptions(precision=2)
    #Code for plotting Confusion matrix
    plt.figure()
    plot_confusion_matrix(confusionMatrix, classes=CLASS_LABELS,
                          title='Confusion matrix, without normalization')
    plt.figure()
    plot_confusion_matrix(confusionMatrix, classes=CLASS_LABELS, normalize=True,
                      title='Normalized confusion matrix')
    plt.show()
    #####################
    score = calculateOverallCostFromConfusionMatrix(confusionMatrix)
    print('Score: ' + str(score))
    return score
    
#calculates the overall cost from the given confusionMAtrix
def calculateOverallCostFromConfusionMatrix(confusionMatrix):
    #cost function from description
    score = confusionMatrix[1][0] * 500 + 10*confusionMatrix[0][1]
    return score

#plots the roc curve for given true labels and the predicted results
def plot_roc(true_lables, predictions):
    true_labels_binarized = label_binarize(true_lables, classes=['neg', 'pos']).flatten()
    predictions_binarized = label_binarize(predictions, classes=['neg', 'pos']).flatten()
    false_positives, true_positives, thresholds = roc_curve(true_labels_binarized, predictions_binarized, pos_label=1)
    roc_auc = auc(false_positives, true_positives)
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positives, true_positives, 'b',
    label='AUC = %0.2f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([-0.1,1.2])
    plt.ylim([-0.1,1.2])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    

#taken from http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
In [6]:
#Loading of all datasets and possible imputations (thus data has to be loaded only once)
####################################################################

#Data, NAN not processed (rows with empty values removed)
trainSet, testSet = loadDataset(None)
trainSet.dropna(inplace=True)
testSet.dropna(inplace=True)
trainSet  = pd.DataFrame(trainSet)
testSet  = pd.DataFrame(testSet)

#Normal data NAN Processed (different strategys used)
meanTrainSet, meanTestSet = loadDataset(NaNSTRATEGY[0])
medianTrainSet, medianTestSet = loadDataset(NaNSTRATEGY[1])
mostFrequentTrainSet, mostFrequentTestSet = loadDataset(NaNSTRATEGY[2])

#Processed Histogram from normal, with NaN processed data
meanTrainSetHist = histogramProcessing(meanTrainSet)
meanTestSetHist = histogramProcessing(meanTestSet)
medianTrainSetHist = histogramProcessing(medianTrainSet)
medianTestSetHist = histogramProcessing(medianTestSet)
mostFrequentTrainSetHist = histogramProcessing(mostFrequentTrainSet)
mostFrequentTestSetHist = histogramProcessing(mostFrequentTestSet)
In [32]:
FtreSelMthd = "SelectKBest"
FtreNo = 20

featureSelectedMeanTrainDf = filterfeatures(meanTrainSet, FtreSelMthd, FtreNo, "Train")
featureSelectedMeanTestDf = filterfeatures(meanTestSet, FtreSelMthd, FtreNo, "Test")

featureSelectedMedianTrainDf = filterfeatures(medianTrainSet, FtreSelMthd, FtreNo, "Train")
featureSelectedMedianTestDf = filterfeatures(medianTestSet, FtreSelMthd, FtreNo, "Test")

featureSelectedFrequentTrainDf = filterfeatures(mostFrequentTrainSet, FtreSelMthd, FtreNo, "Train")
featureSelectedFrequentTestDf = filterfeatures(mostFrequentTestSet, FtreSelMthd, FtreNo, "Test")
In [37]:
#Implementation of scikit Decision Tree
###########################################################


#Use max_depth, and min_sample leafs to prevent overfitting according to http://scikit-learn.org/stable/modules/tree.html#tree 1.10.5
'''
#histogram bin as feature
classifier = DecisionTreeClassifier()#class_weight={'pos':.983, 'neg':.017},max_depth =max_depth, min_samples_leaf =min_samples_leaf)  
classifier.fit(pd.DataFrame(data=meanTrainSet,columns=COLUMNS), pd.DataFrame(data=meanTrainSet,columns=['class']))  
predictions = classifier.predict(pd.DataFrame(data=meanTestSet,columns=COLUMNS))  
score = evaluate(pd.DataFrame(data=meanTestSet,columns=['class']), predictions)
'''

for i in range(5,50,5):
    FtreSelMthd = "SelectKBest"
    FtreNo = i

    featureSelectedMeanTrainDf = filterfeatures(meanTrainSet, FtreSelMthd, FtreNo, "Train")
    display(featureSelectedMeanTrainDf)
    featureSelectedMeanTestDf = filterfeatures(meanTestSet, FtreSelMthd, FtreNo, "Test")

    columnList = featureSelectedMeanTrainDf[featureSelectedMeanTrainDf.columns.difference(['class'])].columns.values

    classifier = DecisionTreeClassifier(class_weight={'pos':.983, 'neg':.017},max_depth =3)  
    classifier.fit(pd.DataFrame(data=featureSelectedMeanTrainDf,columns=columnList), pd.DataFrame(data=featureSelectedMeanTrainDf,columns=['class']))  
    predictions = classifier.predict(pd.DataFrame(data=featureSelectedMeanTestDf,columns=columnList))  
    score = evaluate(pd.DataFrame(data=featureSelectedMeanTestDf,columns=['class']), predictions)



'''
#Hisotgram mean as feature
classifier = DecisionTreeClassifier()#class_weight={'pos':.983, 'neg':.017},max_depth =max_depth, min_samples_leaf =min_samples_leaf)  
classifier.fit(pd.DataFrame(data=meanTrainSetHist,columns=HISTOGRAMMERGEDCOLUMNS), pd.DataFrame(data=meanTrainSetHist,columns=['class']))  
predictions = classifier.predict(pd.DataFrame(data=meanTestSetHist,columns=HISTOGRAMMERGEDCOLUMNS))  
score = evaluate(pd.DataFrame(data=meanTestSetHist,columns=['class']), predictions)


classifier = DecisionTreeClassifier()#class_weight={'pos':.983, 'neg':.017},max_depth =max_depth, min_samples_leaf =min_samples_leaf)  
classifier.fit(pd.DataFrame(data=medianTrainSetHist,columns=HISTOGRAMMERGEDCOLUMNS), pd.DataFrame(data=medianTrainSetHist,columns=['class']))  
predictions = classifier.predict(pd.DataFrame(data=medianTestSetHist,columns=HISTOGRAMMERGEDCOLUMNS))  
score = evaluate(pd.DataFrame(data=medianTestSetHist,columns=['class']), predictions)

classifier = DecisionTreeClassifier(class_weight={'pos':.983, 'neg':.017},max_depth =max_depth, min_samples_leaf =min_samples_leaf)  
classifier.fit(pd.DataFrame(data=mostFrequentTrainSetHist,columns=HISTOGRAMMERGEDCOLUMNS), pd.DataFrame(data=mostFrequentTrainSetHist,columns=['class']))  
predictions = classifier.predict(pd.DataFrame(data=mostFrequentTestSetHist,columns=HISTOGRAMMERGEDCOLUMNS))  
score = evaluate(pd.DataFrame(data=mostFrequentTestSetHist,columns=['class']), predictions)


#Printing of the decision tree
dot_data = tree.export_graphviz(classifier, out_file=None, class_names=CLASS_LABELS, feature_names=COLUMNS, 
                         filled=True, rounded=True,) 
graph = graphviz.Source(dot_data) 
graph.render("DT")

'''
bb_000	bu_000	bv_000	cq_000	dq_000	class
0	6.700214e+06	6.700214e+06	6.700214e+06	6.700214e+06	0.000000e+00	neg
1	3.646660e+06	3.646660e+06	3.646660e+06	3.646660e+06	0.000000e+00	neg
2	2.673338e+06	2.673338e+06	2.673338e+06	2.673338e+06	0.000000e+00	neg
3	2.161400e+04	2.161400e+04	2.161400e+04	2.161400e+04	2.014000e+03	neg
4	4.289260e+06	4.289260e+06	4.289260e+06	4.289260e+06	0.000000e+00	neg
5	2.752812e+06	2.752812e+06	2.752812e+06	2.752812e+06	0.000000e+00	neg
6	1.476400e+04	1.476400e+04	1.476400e+04	1.476400e+04	0.000000e+00	neg
7	6.781222e+06	6.781222e+06	6.781222e+06	6.781222e+06	0.000000e+00	neg
8	7.547346e+06	7.547346e+06	7.547346e+06	7.547346e+06	0.000000e+00	neg
9	7.167172e+06	7.167172e+06	7.167172e+06	7.167172e+06	0.000000e+00	pos
10	2.544730e+06	2.544730e+06	2.544730e+06	2.544730e+06	0.000000e+00	neg
11	3.941672e+06	3.941672e+06	3.941672e+06	3.941672e+06	0.000000e+00	neg
12	1.637280e+05	1.637280e+05	1.637280e+05	1.637280e+05	0.000000e+00	neg
13	9.745646e+06	9.745646e+06	9.745646e+06	9.745646e+06	0.000000e+00	neg
14	1.533608e+06	1.533608e+06	1.533608e+06	1.533608e+06	0.000000e+00	neg
15	5.891000e+04	5.891000e+04	5.891000e+04	5.891000e+04	2.396000e+03	neg
16	1.211186e+07	1.211186e+07	1.211186e+07	1.211186e+07	7.345640e+06	neg
17	7.885000e+04	7.885000e+04	7.885000e+04	7.885000e+04	1.702200e+04	neg
18	3.856600e+04	3.856600e+04	3.856600e+04	3.856600e+04	0.000000e+00	neg
19	1.021640e+05	1.021640e+05	1.021640e+05	1.021640e+05	0.000000e+00	neg
20	2.846732e+06	2.846732e+06	2.846732e+06	2.846732e+06	4.985196e+06	neg
21	1.010200e+04	1.010200e+04	1.010200e+04	1.010200e+04	0.000000e+00	neg
22	1.379007e+07	1.379007e+07	1.379007e+07	1.379007e+07	0.000000e+00	neg
23	3.810041e+07	3.810037e+07	3.810037e+07	3.810037e+07	7.328959e+07	pos
24	4.618048e+06	4.618048e+06	4.618048e+06	4.618048e+06	0.000000e+00	neg
25	2.302530e+06	2.302530e+06	2.302530e+06	2.302530e+06	0.000000e+00	neg
26	8.846680e+05	8.846680e+05	8.846680e+05	8.846680e+05	0.000000e+00	neg
27	3.657128e+06	3.657128e+06	3.657128e+06	3.657128e+06	0.000000e+00	neg
28	1.202580e+05	1.202580e+05	1.202580e+05	1.202580e+05	0.000000e+00	neg
29	1.339568e+06	1.339568e+06	1.339568e+06	1.339568e+06	0.000000e+00	neg
30	5.862092e+06	5.862092e+06	5.862092e+06	5.862092e+06	0.000000e+00	neg
31	1.088980e+05	1.088980e+05	1.088980e+05	1.088980e+05	0.000000e+00	neg
32	1.941192e+06	1.941192e+06	1.941192e+06	1.941192e+06	0.000000e+00	neg
33	3.311044e+06	3.311044e+06	3.311044e+06	3.311044e+06	0.000000e+00	neg
34	6.325600e+04	6.325600e+04	6.325600e+04	6.325600e+04	4.022000e+03	neg
35	1.139480e+05	1.139480e+05	1.139480e+05	1.139480e+05	0.000000e+00	neg
36	5.043418e+06	5.043418e+06	5.043418e+06	5.043418e+06	0.000000e+00	neg
37	4.109878e+06	4.109878e+06	4.109878e+06	4.109878e+06	4.529375e+06	neg
38	1.023440e+05	1.023440e+05	1.023440e+05	1.023440e+05	0.000000e+00	neg
39	2.066414e+06	2.066414e+06	2.066414e+06	2.066414e+06	2.826020e+06	neg
40	9.666000e+03	9.666000e+03	9.666000e+03	9.666000e+03	1.464000e+03	neg
41	1.915990e+06	1.915990e+06	1.915990e+06	1.915990e+06	0.000000e+00	neg
42	1.858140e+05	1.858140e+05	1.858140e+05	1.858140e+05	0.000000e+00	neg
43	1.822600e+04	1.822600e+04	1.822600e+04	1.822600e+04	3.140000e+02	neg
44	5.283528e+06	5.283528e+06	5.283528e+06	5.283528e+06	0.000000e+00	neg
45	1.221820e+05	1.221820e+05	1.221820e+05	1.221820e+05	0.000000e+00	neg
46	6.095632e+06	6.095632e+06	6.095632e+06	6.095632e+06	0.000000e+00	neg
47	1.377600e+04	1.377600e+04	1.377600e+04	1.377600e+04	0.000000e+00	neg
48	2.903060e+06	2.903060e+06	2.903060e+06	2.903060e+06	0.000000e+00	neg
49	4.382426e+06	4.382426e+06	4.382426e+06	4.382426e+06	0.000000e+00	neg
50	1.306540e+05	1.306540e+05	1.306540e+05	1.306540e+05	2.092800e+04	neg
51	4.627600e+04	4.627600e+04	4.627600e+04	4.627600e+04	1.972000e+03	neg
52	2.485912e+06	2.485912e+06	2.485912e+06	2.485912e+06	0.000000e+00	neg
53	1.956400e+04	1.956400e+04	1.956400e+04	1.956400e+04	6.280000e+02	neg
54	4.009948e+06	4.009948e+06	4.009948e+06	4.009948e+06	0.000000e+00	neg
55	2.612306e+06	2.612306e+06	2.612306e+06	2.612306e+06	0.000000e+00	neg
56	4.526177e+06	4.515325e+06	4.515325e+06	4.515325e+06	4.529375e+06	neg
57	2.099600e+04	2.099600e+04	2.099600e+04	2.099600e+04	8.620000e+02	neg
58	1.923400e+04	1.923400e+04	1.923400e+04	1.923400e+04	0.000000e+00	neg
59	4.131540e+05	4.131540e+05	4.131540e+05	4.131540e+05	5.600000e+02	neg
60	4.526177e+06	4.515325e+06	4.515325e+06	4.515325e+06	0.000000e+00	pos
61	2.014564e+06	2.014564e+06	2.014564e+06	2.014564e+06	0.000000e+00	neg
62	2.214628e+06	2.214628e+06	2.214628e+06	2.214628e+06	0.000000e+00	neg
63	2.866270e+06	2.866270e+06	2.866270e+06	2.866270e+06	0.000000e+00	neg
64	2.349256e+06	2.349256e+06	2.349256e+06	2.349256e+06	0.000000e+00	neg
65	2.914632e+06	2.914632e+06	2.914632e+06	2.914632e+06	0.000000e+00	neg
66	2.823810e+06	2.823810e+06	2.823810e+06	2.823810e+06	0.000000e+00	neg
67	1.016740e+05	1.016740e+05	1.016740e+05	1.016740e+05	2.414200e+04	neg
68	1.573600e+04	1.573600e+04	1.573600e+04	1.573600e+04	3.000000e+02	neg
69	2.914718e+06	2.914718e+06	2.914718e+06	2.914718e+06	0.000000e+00	neg
70	2.987100e+06	2.987100e+06	2.987100e+06	2.987100e+06	0.000000e+00	neg
71	2.412380e+05	2.412380e+05	2.412380e+05	2.412380e+05	4.602560e+05	neg
72	3.485000e+05	3.485000e+05	3.485000e+05	3.485000e+05	4.529375e+06	neg
73	6.886766e+06	6.886766e+06	6.886766e+06	6.886766e+06	1.939267e+07	neg
74	3.498004e+06	3.498004e+06	3.498004e+06	3.498004e+06	2.846399e+08	neg
75	5.374914e+06	5.374914e+06	5.374914e+06	5.374914e+06	0.000000e+00	neg
76	2.088650e+06	2.088650e+06	2.088650e+06	2.088650e+06	0.000000e+00	neg
77	1.722940e+05	1.722940e+05	1.722940e+05	1.722940e+05	0.000000e+00	neg
78	1.098240e+05	1.098240e+05	1.098240e+05	1.098240e+05	0.000000e+00	neg
79	3.380020e+06	3.380020e+06	3.380020e+06	3.380020e+06	0.000000e+00	neg
80	2.972948e+06	2.972948e+06	2.972948e+06	2.972948e+06	0.000000e+00	neg
81	8.303800e+04	8.303800e+04	8.303800e+04	8.303800e+04	0.000000e+00	neg
82	1.393820e+05	1.393820e+05	1.393820e+05	1.393820e+05	6.634000e+03	neg
83	2.038600e+04	2.038600e+04	2.038600e+04	2.038600e+04	2.400000e+02	neg
84	2.011600e+04	2.011600e+04	2.011600e+04	2.011600e+04	1.228000e+03	neg
85	8.357000e+04	8.357000e+04	8.357000e+04	8.357000e+04	0.000000e+00	neg
86	1.421878e+07	1.421878e+07	1.421878e+07	1.421878e+07	4.529375e+06	neg
87	1.322800e+05	1.322800e+05	1.322800e+05	1.322800e+05	0.000000e+00	neg
88	5.203200e+04	5.203200e+04	5.203200e+04	5.203200e+04	0.000000e+00	neg
89	1.842540e+05	1.842540e+05	1.842540e+05	1.842540e+05	0.000000e+00	neg
90	9.974600e+04	9.974600e+04	9.974600e+04	9.974600e+04	0.000000e+00	neg
91	1.519746e+06	1.519746e+06	1.519746e+06	1.519746e+06	0.000000e+00	neg
92	3.138852e+06	3.138852e+06	3.138852e+06	3.138852e+06	0.000000e+00	neg
93	8.452086e+06	8.452086e+06	8.452086e+06	8.452086e+06	0.000000e+00	neg
94	3.011846e+06	3.011846e+06	3.011846e+06	3.011846e+06	0.000000e+00	neg
95	2.515250e+06	2.515250e+06	2.515250e+06	2.515250e+06	0.000000e+00	neg
96	8.140320e+05	8.140320e+05	8.140320e+05	8.140320e+05	3.715340e+05	neg
97	2.698290e+06	2.698290e+06	2.698290e+06	2.698290e+06	0.000000e+00	neg
98	2.186022e+06	2.186022e+06	2.186022e+06	2.186022e+06	0.000000e+00	neg
99	2.567858e+06	2.567858e+06	2.567858e+06	2.567858e+06	0.000000e+00	neg
100	1.565900e+05	1.565900e+05	1.565900e+05	1.565900e+05	0.000000e+00	neg
101	5.206532e+06	5.206532e+06	5.206532e+06	5.206532e+06	0.000000e+00	neg
102	1.607000e+04	1.607000e+04	1.607000e+04	1.607000e+04	0.000000e+00	neg
103	4.056980e+05	4.056980e+05	4.056980e+05	4.056980e+05	1.354320e+05	neg
104	1.226086e+07	1.226086e+07	1.226086e+07	1.226086e+07	4.529375e+06	neg
105	1.720160e+05	1.720160e+05	1.720160e+05	1.720160e+05	0.000000e+00	neg
106	3.700544e+06	3.700544e+06	3.700544e+06	3.700544e+06	7.216672e+06	neg
107	1.825408e+06	1.825408e+06	1.825408e+06	1.825408e+06	0.000000e+00	neg
108	5.790200e+04	5.790200e+04	5.790200e+04	5.790200e+04	7.280000e+02	neg
109	2.441600e+04	2.441600e+04	2.441600e+04	2.441600e+04	7.800000e+01	neg
110	1.833200e+04	1.833200e+04	1.833200e+04	1.833200e+04	1.398000e+03	neg
111	1.373317e+07	1.373317e+07	1.373317e+07	1.373317e+07	4.529375e+06	neg
112	3.640478e+06	3.640478e+06	3.640478e+06	3.640478e+06	0.000000e+00	neg
113	2.635510e+06	2.635510e+06	2.635510e+06	2.635510e+06	0.000000e+00	neg
114	2.649982e+06	2.649982e+06	2.649982e+06	2.649982e+06	0.000000e+00	neg
115	4.692868e+07	4.692868e+07	4.692868e+07	4.692868e+07	4.529375e+06	pos
116	1.039620e+05	1.039620e+05	1.039620e+05	1.039620e+05	0.000000e+00	neg
117	1.696600e+04	1.696600e+04	1.696600e+04	1.696600e+04	0.000000e+00	neg
118	2.407712e+06	2.407712e+06	2.407712e+06	2.407712e+06	0.000000e+00	neg
119	1.946380e+05	1.946380e+05	1.946380e+05	1.946380e+05	0.000000e+00	neg
120	7.593800e+04	7.593800e+04	7.593800e+04	7.593800e+04	0.000000e+00	neg
121	8.989374e+06	8.989374e+06	8.989374e+06	8.989374e+06	4.529375e+06	neg
122	2.090000e+04	2.090000e+04	2.090000e+04	2.090000e+04	1.212000e+03	neg
123	4.865336e+06	4.865336e+06	4.865336e+06	4.865336e+06	0.000000e+00	neg
124	6.173314e+06	6.173314e+06	6.173314e+06	6.173314e+06	0.000000e+00	neg
125	7.999200e+04	7.999200e+04	7.999200e+04	7.999200e+04	0.000000e+00	neg
126	1.776110e+06	1.776110e+06	1.776110e+06	1.776110e+06	4.529375e+06	neg
127	5.517820e+05	5.517820e+05	5.517820e+05	5.517820e+05	0.000000e+00	neg
128	6.700650e+06	6.700650e+06	6.700650e+06	6.700650e+06	0.000000e+00	neg
129	7.280180e+05	7.280180e+05	7.280180e+05	7.280180e+05	4.529375e+06	neg
130	1.013800e+04	1.013800e+04	1.013800e+04	1.013800e+04	4.162000e+03	neg
131	9.947600e+04	9.947600e+04	9.947600e+04	9.947600e+04	0.000000e+00	neg
132	1.247580e+06	1.247580e+06	1.247580e+06	1.247580e+06	4.956009e+07	neg
133	9.097800e+04	9.097800e+04	9.097800e+04	9.097800e+04	2.352800e+04	neg
134	2.461180e+05	2.461180e+05	2.461180e+05	2.461180e+05	0.000000e+00	neg
135	8.529151e+07	4.515325e+06	4.515325e+06	4.515325e+06	4.529375e+06	pos
136	7.361919e+07	7.361919e+07	7.361919e+07	7.361919e+07	0.000000e+00	neg
137	7.981600e+04	7.981600e+04	7.981600e+04	7.981600e+04	0.000000e+00	neg
138	5.876386e+06	5.876386e+06	5.876386e+06	5.876386e+06	0.000000e+00	neg
139	2.766320e+05	2.766320e+05	2.766320e+05	2.766320e+05	1.354800e+04	neg
140	8.886502e+07	8.886502e+07	8.886502e+07	8.886502e+07	4.529375e+06	neg
141	1.836480e+05	1.836480e+05	1.836480e+05	1.836480e+05	0.000000e+00	neg
142	2.158680e+07	2.158680e+07	2.158680e+07	2.158680e+07	0.000000e+00	neg
143	1.680298e+07	1.680298e+07	1.680298e+07	1.680298e+07	0.000000e+00	neg
144	7.795200e+04	7.795200e+04	7.795200e+04	7.795200e+04	0.000000e+00	neg
145	5.080400e+04	5.080400e+04	5.080400e+04	5.080400e+04	0.000000e+00	neg
146	2.562200e+05	2.562200e+05	2.562200e+05	2.562200e+05	2.226000e+03	neg
147	3.623564e+06	3.623564e+06	3.623564e+06	3.623564e+06	0.000000e+00	neg
148	2.690508e+06	2.690508e+06	2.690508e+06	2.690508e+06	0.000000e+00	neg
149	2.210070e+06	2.210070e+06	2.210070e+06	2.210070e+06	0.000000e+00	neg
150	7.803480e+05	7.803480e+05	7.803480e+05	7.803480e+05	0.000000e+00	neg
151	1.881986e+06	1.881986e+06	1.881986e+06	1.881986e+06	4.529375e+06	neg
152	3.580642e+06	3.580642e+06	3.580642e+06	3.580642e+06	0.000000e+00	neg
153	2.053722e+06	2.053722e+06	2.053722e+06	2.053722e+06	4.529375e+06	neg
154	4.958228e+06	4.958228e+06	4.958228e+06	4.958228e+06	0.000000e+00	neg
155	1.619280e+05	1.619280e+05	1.619280e+05	1.619280e+05	0.000000e+00	neg
156	1.226902e+08	1.226902e+08	1.226902e+08	1.226902e+08	1.341170e+08	pos
157	2.157828e+06	2.157828e+06	2.157828e+06	2.157828e+06	0.000000e+00	neg
158	2.841400e+04	2.841400e+04	2.841400e+04	2.841400e+04	8.440000e+02	neg
159	1.288260e+05	1.288260e+05	1.288260e+05	1.288260e+05	0.000000e+00	neg
160	3.179322e+06	3.179322e+06	3.179322e+06	3.179322e+06	0.000000e+00	neg
161	7.063886e+06	7.063886e+06	7.063886e+06	7.063886e+06	0.000000e+00	neg
162	6.502200e+04	6.502200e+04	6.502200e+04	6.502200e+04	0.000000e+00	neg
163	3.128281e+07	3.128281e+07	3.128281e+07	3.128281e+07	0.000000e+00	neg
164	7.790126e+06	7.790126e+06	7.790126e+06	7.790126e+06	1.579550e+07	neg
165	1.148160e+05	1.148160e+05	1.148160e+05	1.148160e+05	0.000000e+00	neg
166	2.227752e+06	2.227752e+06	2.227752e+06	2.227752e+06	0.000000e+00	neg
167	2.280754e+06	2.280754e+06	2.280754e+06	2.280754e+06	0.000000e+00	neg
168	1.827540e+05	1.827540e+05	1.827540e+05	1.827540e+05	0.000000e+00	neg
169	1.547500e+05	1.547500e+05	1.547500e+05	1.547500e+05	0.000000e+00	neg
170	2.618644e+06	2.618644e+06	2.618644e+06	2.618644e+06	0.000000e+00	neg
171	3.152078e+06	3.152078e+06	3.152078e+06	3.152078e+06	0.000000e+00	neg
172	1.433200e+05	1.433200e+05	1.433200e+05	1.433200e+05	0.000000e+00	neg
173	6.874600e+04	6.874600e+04	6.874600e+04	6.874600e+04	0.000000e+00	neg
174	2.740470e+06	2.740470e+06	2.740470e+06	2.740470e+06	0.000000e+00	neg
175	1.291847e+07	1.291847e+07	1.291847e+07	1.291847e+07	4.529375e+06	neg
176	1.783602e+06	1.783602e+06	1.783602e+06	1.783602e+06	0.000000e+00	neg
177	3.325019e+07	3.325019e+07	3.325019e+07	3.325019e+07	4.536022e+06	pos
178	1.797560e+05	1.797560e+05	1.797560e+05	1.797560e+05	3.379800e+04	neg
179	1.108075e+07	1.108075e+07	1.108075e+07	1.108075e+07	7.053478e+06	neg
180	4.079400e+04	4.079400e+04	4.079400e+04	4.079400e+04	8.080000e+03	neg
181	1.236860e+05	1.236860e+05	1.236860e+05	1.236860e+05	0.000000e+00	neg
182	2.710898e+06	2.710898e+06	2.710898e+06	2.710898e+06	0.000000e+00	neg
183	4.545146e+06	4.545146e+06	4.545146e+06	4.545146e+06	0.000000e+00	neg
184	2.295462e+06	2.295462e+06	2.295462e+06	2.295462e+06	0.000000e+00	neg
185	7.509720e+06	7.509720e+06	7.509720e+06	7.509720e+06	1.440366e+07	neg
186	2.128960e+05	2.128960e+05	2.128960e+05	2.128960e+05	0.000000e+00	neg
187	2.484102e+06	2.484102e+06	2.484102e+06	2.484102e+06	0.000000e+00	neg
188	6.905566e+06	6.905566e+06	6.905566e+06	6.905566e+06	0.000000e+00	neg
189	1.706400e+04	1.706400e+04	1.706400e+04	1.706400e+04	0.000000e+00	neg
190	3.017064e+06	3.017064e+06	3.017064e+06	3.017064e+06	0.000000e+00	neg
191	2.485380e+05	2.485380e+05	2.485380e+05	2.485380e+05	0.000000e+00	neg
192	3.262134e+06	3.262134e+06	3.262134e+06	3.262134e+06	3.796348e+06	neg
193	4.745308e+06	4.745308e+06	4.745308e+06	4.745308e+06	0.000000e+00	neg
194	2.782184e+06	2.782184e+06	2.782184e+06	2.782184e+06	0.000000e+00	neg
195	1.315320e+05	1.315320e+05	1.315320e+05	1.315320e+05	0.000000e+00	neg
196	2.451308e+06	2.451308e+06	2.451308e+06	2.451308e+06	0.000000e+00	neg
197	3.949914e+06	3.949914e+06	3.949914e+06	3.949914e+06	3.975892e+06	neg
198	2.424176e+06	2.424176e+06	2.424176e+06	2.424176e+06	6.179824e+06	neg
199	1.434360e+05	1.434360e+05	1.434360e+05	1.434360e+05	0.000000e+00	neg
200	7.482921e+07	7.482921e+07	7.482921e+07	7.482921e+07	7.967351e+07	neg
201	9.689600e+04	9.689600e+04	9.689600e+04	9.689600e+04	1.465000e+04	neg
202	2.633861e+07	2.633861e+07	2.633861e+07	2.633861e+07	1.996512e+07	neg
203	3.657278e+06	3.657278e+06	3.657278e+06	3.657278e+06	6.352630e+06	neg
204	3.502678e+06	3.502678e+06	3.502678e+06	3.502678e+06	5.706866e+06	neg
205	2.202812e+06	2.202812e+06	2.202812e+06	2.202812e+06	0.000000e+00	neg
206	2.317352e+06	2.317352e+06	2.317352e+06	2.317352e+06	0.000000e+00	neg
207	8.004336e+06	8.004336e+06	8.004336e+06	8.004336e+06	2.450855e+07	neg
208	4.064988e+06	4.064988e+06	4.064988e+06	4.064988e+06	8.264828e+06	neg
209	9.633600e+04	9.633600e+04	9.633600e+04	9.633600e+04	0.000000e+00	neg
210	5.332609e+07	5.332609e+07	5.332609e+07	5.332609e+07	4.529375e+06	pos
211	3.790204e+06	3.790204e+06	3.790204e+06	3.790204e+06	0.000000e+00	neg
212	4.718800e+04	4.718800e+04	4.718800e+04	4.718800e+04	3.600000e+02	neg
213	2.277080e+05	2.277080e+05	2.277080e+05	2.277080e+05	1.002400e+04	neg
214	4.957608e+06	4.957608e+06	4.957608e+06	4.957608e+06	4.529375e+06	neg
215	6.787800e+04	6.787800e+04	6.787800e+04	6.787800e+04	0.000000e+00	neg
216	3.199600e+04	3.199600e+04	3.199600e+04	3.199600e+04	0.000000e+00	neg
217	2.487648e+06	2.487648e+06	2.487648e+06	2.487648e+06	2.695804e+06	neg
218	1.689400e+04	1.689400e+04	1.689400e+04	1.689400e+04	0.000000e+00	neg
219	4.835260e+05	4.835260e+05	4.835260e+05	4.835260e+05	0.000000e+00	neg
220	1.086740e+05	1.086740e+05	1.086740e+05	1.086740e+05	0.000000e+00	neg
221	2.109744e+06	2.109744e+06	2.109744e+06	2.109744e+06	4.529375e+06	neg
222	2.613072e+06	2.613072e+06	2.613072e+06	2.613072e+06	0.000000e+00	neg
223	9.609798e+06	9.609798e+06	9.609798e+06	9.609798e+06	0.000000e+00	neg
224	3.828114e+06	3.828114e+06	3.828114e+06	3.828114e+06	0.000000e+00	neg
225	1.008918e+07	1.008918e+07	1.008918e+07	1.008918e+07	1.851548e+07	neg
226	2.797080e+06	2.797080e+06	2.797080e+06	2.797080e+06	0.000000e+00	neg
227	5.867462e+06	5.867462e+06	5.867462e+06	5.867462e+06	0.000000e+00	neg
228	2.007800e+04	2.007800e+04	2.007800e+04	2.007800e+04	0.000000e+00	neg
229	2.895154e+06	2.890546e+06	2.895154e+06	2.895154e+06	0.000000e+00	neg
230	9.082708e+06	9.082708e+06	9.082708e+06	9.082708e+06	0.000000e+00	neg
231	3.456468e+06	3.456468e+06	3.456468e+06	3.456468e+06	0.000000e+00	neg
232	4.137492e+06	4.137492e+06	4.137492e+06	4.137492e+06	8.115686e+06	neg
233	3.620800e+04	3.620800e+04	3.620800e+04	3.620800e+04	0.000000e+00	neg
234	4.474272e+06	4.474272e+06	4.474272e+06	4.474272e+06	0.000000e+00	neg
235	2.829166e+06	2.829166e+06	2.829166e+06	2.829166e+06	0.000000e+00	neg
236	3.395476e+06	3.395476e+06	3.395476e+06	3.395476e+06	0.000000e+00	neg
237	2.429098e+06	2.429098e+06	2.429098e+06	2.429098e+06	4.529375e+06	neg
238	8.503800e+04	8.503800e+04	8.503800e+04	8.503800e+04	0.000000e+00	neg
239	1.827678e+06	1.827678e+06	1.827678e+06	1.827678e+06	0.000000e+00	neg
240	1.438880e+05	1.438880e+05	1.438880e+05	1.438880e+05	0.000000e+00	neg
241	1.203462e+06	1.203462e+06	1.203462e+06	1.203462e+06	4.529375e+06	neg
242	6.598854e+06	6.598854e+06	6.598854e+06	6.598854e+06	0.000000e+00	neg
243	1.746600e+05	1.746600e+05	1.746600e+05	1.746600e+05	0.000000e+00	neg
244	1.736420e+05	1.736420e+05	1.736420e+05	1.736420e+05	6.348200e+04	neg
245	2.014800e+04	2.014800e+04	2.014800e+04	2.014800e+04	0.000000e+00	neg
246	1.521400e+05	1.521400e+05	1.521400e+05	1.521400e+05	2.245600e+04	neg
247	4.700902e+06	4.700902e+06	4.700902e+06	4.700902e+06	0.000000e+00	neg
248	4.526177e+06	4.515325e+06	4.515325e+06	4.515325e+06	0.000000e+00	neg
249	2.675344e+06	2.675344e+06	2.675344e+06	2.675344e+06	0.000000e+00	neg
...	...	...	...	...	...	...
59750	4.582600e+04	4.582600e+04	4.582600e+04	4.582600e+04	4.529375e+06	neg
59751	1.747800e+04	1.747800e+04	1.747800e+04	1.747800e+04	0.000000e+00	neg
59752	6.905320e+05	6.905320e+05	6.905320e+05	6.905320e+05	0.000000e+00	neg
59753	1.661400e+04	1.661400e+04	1.661400e+04	1.661400e+04	2.780000e+02	neg
59754	2.697000e+04	2.697000e+04	2.697000e+04	2.697000e+04	0.000000e+00	neg
59755	2.184400e+04	2.184400e+04	2.184400e+04	2.184400e+04	0.000000e+00	neg
59756	1.252040e+05	1.252040e+05	1.252040e+05	1.252040e+05	2.616200e+04	neg
59757	1.868102e+06	1.868102e+06	1.868102e+06	1.868102e+06	0.000000e+00	neg
59758	7.025800e+04	7.025800e+04	7.025800e+04	7.025800e+04	0.000000e+00	neg
59759	1.939400e+04	1.939400e+04	1.939400e+04	1.939400e+04	0.000000e+00	neg
59760	2.362660e+06	2.362660e+06	2.362660e+06	2.362660e+06	0.000000e+00	neg
59761	2.233190e+06	2.233190e+06	2.233190e+06	2.233190e+06	0.000000e+00	neg
59762	3.145368e+06	3.145368e+06	3.145368e+06	3.145368e+06	0.000000e+00	neg
59763	1.140973e+07	1.140973e+07	1.140973e+07	1.140973e+07	1.511409e+07	neg
59764	5.114380e+06	5.114380e+06	5.114380e+06	5.114380e+06	0.000000e+00	neg
59765	1.376400e+05	1.376400e+05	1.376400e+05	1.376400e+05	2.831000e+04	neg
59766	7.660720e+06	7.660720e+06	7.660720e+06	7.660720e+06	7.727996e+06	neg
59767	4.436870e+06	4.436870e+06	4.436870e+06	4.436870e+06	0.000000e+00	neg
59768	3.017000e+04	3.017000e+04	3.017000e+04	3.017000e+04	1.752000e+03	neg
59769	3.213564e+07	3.213564e+07	3.213564e+07	3.213564e+07	0.000000e+00	pos
59770	8.479048e+06	8.479048e+06	8.479048e+06	8.479048e+06	0.000000e+00	neg
59771	1.059200e+05	1.059200e+05	1.059200e+05	1.059200e+05	0.000000e+00	neg
59772	9.367454e+06	9.367454e+06	9.367454e+06	9.367454e+06	4.529375e+06	neg
59773	7.873000e+04	7.873000e+04	7.873000e+04	7.873000e+04	0.000000e+00	neg
59774	1.530920e+05	1.530920e+05	1.530920e+05	1.530920e+05	0.000000e+00	neg
59775	2.940468e+06	2.940468e+06	2.940468e+06	2.940468e+06	0.000000e+00	neg
59776	1.434400e+05	1.434400e+05	1.434400e+05	1.434400e+05	0.000000e+00	neg
59777	1.629800e+04	1.629800e+04	1.629800e+04	1.629800e+04	2.660000e+02	neg
59778	2.404780e+05	2.404780e+05	2.404780e+05	2.404780e+05	0.000000e+00	neg
59779	6.703800e+04	6.703800e+04	6.703800e+04	6.703800e+04	0.000000e+00	neg
59780	2.435080e+06	2.435080e+06	2.435080e+06	2.435080e+06	0.000000e+00	neg
59781	6.325200e+04	6.325200e+04	6.325200e+04	6.325200e+04	6.896000e+03	neg
59782	2.612818e+06	2.612818e+06	2.612818e+06	2.612818e+06	0.000000e+00	neg
59783	7.450200e+04	7.450200e+04	7.450200e+04	7.450200e+04	2.400000e+03	neg
59784	1.774800e+04	1.774800e+04	1.774800e+04	1.774800e+04	0.000000e+00	neg
59785	1.443552e+06	1.443552e+06	1.443552e+06	1.443552e+06	0.000000e+00	neg
59786	3.386912e+06	3.386912e+06	3.386912e+06	3.386912e+06	0.000000e+00	neg
59787	5.310428e+06	5.310428e+06	5.310428e+06	5.310428e+06	0.000000e+00	neg
59788	1.032070e+07	1.032070e+07	1.032070e+07	1.032070e+07	1.184058e+07	neg
59789	1.133331e+07	1.133331e+07	1.133331e+07	1.133331e+07	4.529375e+06	neg
59790	2.898400e+04	2.898400e+04	2.898400e+04	2.898400e+04	7.880000e+02	neg
59791	4.281934e+06	4.281934e+06	4.281934e+06	4.281934e+06	0.000000e+00	neg
59792	5.793598e+06	5.793598e+06	5.793598e+06	5.793598e+06	0.000000e+00	neg
59793	4.105400e+04	4.105400e+04	4.105400e+04	4.105400e+04	3.024000e+03	neg
59794	1.731000e+04	1.731000e+04	1.731000e+04	1.731000e+04	6.600000e+02	neg
59795	1.242390e+07	1.242390e+07	1.242390e+07	1.242390e+07	0.000000e+00	neg
59796	1.337260e+05	1.337260e+05	1.337260e+05	1.337260e+05	7.518400e+04	neg
59797	1.546400e+04	1.546400e+04	1.546400e+04	1.546400e+04	1.174000e+03	neg
59798	2.406596e+06	2.406596e+06	2.406596e+06	2.406596e+06	0.000000e+00	neg
59799	3.012812e+06	3.012812e+06	3.012812e+06	3.012812e+06	0.000000e+00	neg
59800	1.324000e+04	1.324000e+04	1.324000e+04	1.324000e+04	4.040000e+02	neg
59801	2.442080e+05	2.442080e+05	2.442080e+05	2.442080e+05	4.257440e+05	neg
59802	1.751580e+05	1.751580e+05	1.751580e+05	1.751580e+05	0.000000e+00	neg
59803	3.687116e+06	3.687116e+06	3.687116e+06	3.687116e+06	0.000000e+00	neg
59804	4.070028e+06	4.070028e+06	4.070028e+06	4.070028e+06	0.000000e+00	neg
59805	3.438770e+06	3.438770e+06	3.438770e+06	3.438770e+06	0.000000e+00	neg
59806	4.052196e+06	4.052196e+06	4.052196e+06	4.052196e+06	0.000000e+00	neg
59807	3.025200e+04	3.025200e+04	3.025200e+04	3.025200e+04	2.904000e+03	neg
59808	1.227940e+05	1.227940e+05	1.227940e+05	1.227940e+05	0.000000e+00	neg
59809	4.582826e+06	4.582826e+06	4.582826e+06	4.582826e+06	6.948316e+06	neg
59810	4.365400e+04	4.365400e+04	4.365400e+04	4.365400e+04	1.800000e+01	neg
59811	1.068860e+05	1.068860e+05	1.068860e+05	1.068860e+05	1.988400e+04	neg
59812	3.262200e+04	3.262200e+04	3.262200e+04	3.262200e+04	0.000000e+00	neg
59813	2.302000e+04	2.302000e+04	2.302000e+04	2.302000e+04	1.668000e+03	neg
59814	1.709866e+06	1.709866e+06	1.709866e+06	1.709866e+06	0.000000e+00	neg
59815	5.044416e+06	5.044416e+06	5.044416e+06	5.044416e+06	0.000000e+00	neg
59816	3.254028e+06	3.254028e+06	3.254028e+06	3.254028e+06	0.000000e+00	neg
59817	1.569859e+07	1.569859e+07	1.569859e+07	1.569859e+07	0.000000e+00	neg
59818	1.649800e+04	1.649800e+04	1.649800e+04	1.649800e+04	2.680000e+02	neg
59819	1.382800e+04	1.382800e+04	1.382800e+04	1.382800e+04	0.000000e+00	neg
59820	2.686499e+07	2.686499e+07	2.686499e+07	2.686499e+07	0.000000e+00	neg
59821	2.043744e+06	2.043744e+06	2.043744e+06	2.043744e+06	0.000000e+00	neg
59822	1.544000e+04	1.544000e+04	1.544000e+04	1.544000e+04	0.000000e+00	neg
59823	2.736000e+04	2.736000e+04	2.736000e+04	2.736000e+04	0.000000e+00	neg
59824	1.253732e+06	1.253732e+06	1.253732e+06	1.253732e+06	0.000000e+00	neg
59825	2.514700e+06	2.514700e+06	2.514700e+06	2.514700e+06	0.000000e+00	neg
59826	8.933000e+04	8.933000e+04	8.933000e+04	8.933000e+04	3.100000e+03	neg
59827	9.097952e+06	9.097952e+06	9.097952e+06	9.097952e+06	0.000000e+00	neg
59828	4.248530e+06	4.248530e+06	4.248530e+06	4.248530e+06	0.000000e+00	neg
59829	4.703400e+04	4.703400e+04	4.703400e+04	4.703400e+04	0.000000e+00	neg
59830	1.929760e+05	1.929760e+05	1.929760e+05	1.929760e+05	0.000000e+00	neg
59831	2.884326e+06	2.884326e+06	2.884326e+06	2.884326e+06	0.000000e+00	neg
59832	4.727920e+05	4.727920e+05	4.727920e+05	4.727920e+05	0.000000e+00	neg
59833	4.505066e+06	4.505066e+06	4.505066e+06	4.505066e+06	6.154236e+06	neg
59834	1.924998e+06	1.924998e+06	1.924998e+06	1.924998e+06	0.000000e+00	neg
59835	2.930000e+04	2.930000e+04	2.930000e+04	2.930000e+04	2.322000e+03	neg
59836	3.244200e+04	3.244200e+04	3.244200e+04	3.244200e+04	0.000000e+00	neg
59837	5.205136e+06	5.205136e+06	5.205136e+06	5.205136e+06	0.000000e+00	neg
59838	3.112782e+06	3.112782e+06	3.112782e+06	3.112782e+06	0.000000e+00	neg
59839	9.983000e+04	9.983000e+04	9.983000e+04	9.983000e+04	0.000000e+00	neg
59840	3.265022e+06	3.265022e+06	3.265022e+06	3.265022e+06	0.000000e+00	neg
59841	2.192278e+06	2.192278e+06	2.192278e+06	2.192278e+06	0.000000e+00	neg
59842	2.614992e+06	2.614992e+06	2.614992e+06	2.614992e+06	0.000000e+00	neg
59843	1.440904e+07	1.440904e+07	1.440904e+07	1.440904e+07	1.203186e+07	neg
59844	2.458560e+05	2.458560e+05	2.458560e+05	2.458560e+05	0.000000e+00	neg
59845	6.852994e+06	6.852994e+06	6.852994e+06	6.852994e+06	0.000000e+00	neg
59846	3.137617e+07	3.137617e+07	3.137617e+07	3.137617e+07	4.529375e+06	neg
59847	1.375020e+05	1.375020e+05	1.375020e+05	1.375020e+05	4.529375e+06	neg
59848	2.936972e+06	2.936972e+06	2.936972e+06	2.936972e+06	0.000000e+00	neg
59849	1.474030e+06	1.474030e+06	1.474030e+06	1.474030e+06	0.000000e+00	neg
59850	3.924588e+06	3.924588e+06	3.924588e+06	3.924588e+06	0.000000e+00	neg
59851	2.506528e+06	2.506528e+06	2.506528e+06	2.506528e+06	0.000000e+00	neg
59852	1.299400e+04	1.299400e+04	1.299400e+04	1.299400e+04	1.560000e+02	neg
59853	2.845438e+06	2.845438e+06	2.845438e+06	2.845438e+06	0.000000e+00	neg
59854	5.180422e+06	5.180422e+06	5.180422e+06	5.180422e+06	0.000000e+00	neg
59855	3.232606e+06	3.232606e+06	3.232606e+06	3.232606e+06	5.403664e+06	neg
59856	3.327720e+06	3.327720e+06	3.327720e+06	3.327720e+06	0.000000e+00	neg
59857	5.206036e+06	5.206036e+06	5.206036e+06	5.206036e+06	0.000000e+00	neg
59858	2.700800e+04	2.700800e+04	2.700800e+04	2.700800e+04	0.000000e+00	neg
59859	5.515128e+06	5.515128e+06	5.515128e+06	5.515128e+06	0.000000e+00	neg
59860	2.898202e+06	2.898202e+06	2.898202e+06	2.898202e+06	0.000000e+00	neg
59861	2.473128e+06	2.473128e+06	2.473128e+06	2.473128e+06	0.000000e+00	neg
59862	7.736400e+04	7.736400e+04	7.736400e+04	7.736400e+04	6.187200e+04	neg
59863	1.861600e+04	1.861600e+04	1.861600e+04	1.861600e+04	0.000000e+00	neg
59864	4.526177e+06	4.515325e+06	4.515325e+06	4.515325e+06	4.529375e+06	neg
59865	4.526177e+06	4.515325e+06	4.515325e+06	4.515325e+06	2.649060e+05	neg
59866	1.964519e+07	1.964519e+07	1.964519e+07	1.964519e+07	4.529375e+06	neg
59867	4.219942e+06	4.219942e+06	4.219942e+06	4.219942e+06	0.000000e+00	neg
59868	1.739501e+07	1.739501e+07	1.739501e+07	1.739501e+07	0.000000e+00	neg
59869	2.668424e+06	2.668424e+06	2.668424e+06	2.668424e+06	0.000000e+00	neg
59870	1.187716e+07	1.187716e+07	1.187716e+07	1.187716e+07	1.910598e+07	neg
59871	6.715306e+06	6.715306e+06	6.715306e+06	6.715306e+06	0.000000e+00	neg
59872	3.580236e+06	3.580236e+06	3.580236e+06	3.580236e+06	0.000000e+00	neg
59873	1.465392e+06	1.465392e+06	1.465392e+06	1.465392e+06	0.000000e+00	neg
59874	1.056060e+05	1.056060e+05	1.056060e+05	1.056060e+05	0.000000e+00	neg
59875	2.727440e+05	2.727440e+05	2.727440e+05	2.727440e+05	0.000000e+00	neg
59876	1.235622e+06	1.235622e+06	1.235622e+06	1.235622e+06	0.000000e+00	neg
59877	3.229356e+06	3.229356e+06	3.229356e+06	3.229356e+06	3.778128e+06	neg
59878	7.423000e+04	7.423000e+04	7.423000e+04	7.423000e+04	0.000000e+00	neg
59879	5.388040e+05	5.388040e+05	5.388040e+05	5.388040e+05	0.000000e+00	neg
59880	2.450760e+05	2.450760e+05	2.450760e+05	2.450760e+05	4.944600e+04	neg
59881	2.570466e+06	2.570466e+06	2.570466e+06	2.570466e+06	0.000000e+00	neg
59882	2.196298e+06	2.196298e+06	2.196298e+06	2.196298e+06	0.000000e+00	neg
59883	3.865642e+06	3.865642e+06	3.865642e+06	3.865642e+06	0.000000e+00	neg
59884	3.468958e+06	3.468958e+06	3.468958e+06	3.468958e+06	0.000000e+00	neg
59885	1.796240e+05	1.796240e+05	1.796240e+05	1.796240e+05	5.076000e+04	neg
59886	4.291350e+06	4.291350e+06	4.291350e+06	4.291350e+06	0.000000e+00	neg
59887	5.194441e+07	5.194441e+07	5.194441e+07	5.194441e+07	0.000000e+00	neg
59888	2.304322e+06	2.304322e+06	2.304322e+06	2.304322e+06	0.000000e+00	neg
59889	2.658864e+06	2.658864e+06	2.658864e+06	2.658864e+06	0.000000e+00	neg
59890	8.550200e+04	8.550200e+04	8.550200e+04	8.550200e+04	0.000000e+00	neg
59891	2.850400e+04	2.850400e+04	2.850400e+04	2.850400e+04	0.000000e+00	neg
59892	2.050160e+06	2.050160e+06	2.050160e+06	2.050160e+06	0.000000e+00	neg
59893	2.264820e+06	2.264820e+06	2.264820e+06	2.264820e+06	0.000000e+00	neg
59894	3.200192e+06	3.200192e+06	3.200192e+06	3.200192e+06	0.000000e+00	neg
59895	2.060962e+06	2.060962e+06	2.060962e+06	2.060962e+06	0.000000e+00	neg
59896	2.063866e+06	2.063866e+06	2.063866e+06	2.063866e+06	0.000000e+00	neg
59897	2.800800e+04	2.800800e+04	2.800800e+04	2.800800e+04	8.000000e+01	neg
59898	2.735590e+06	2.735590e+06	2.735590e+06	2.735590e+06	0.000000e+00	neg
59899	1.088600e+04	1.088600e+04	1.088600e+04	1.088600e+04	0.000000e+00	neg
59900	3.151648e+06	3.151648e+06	3.151648e+06	3.151648e+06	0.000000e+00	neg
59901	1.564840e+05	1.564840e+05	1.564840e+05	1.564840e+05	0.000000e+00	neg
59902	7.309986e+06	7.309986e+06	7.309986e+06	7.309986e+06	0.000000e+00	neg
59903	2.372172e+06	2.372172e+06	2.372172e+06	2.372172e+06	0.000000e+00	neg
59904	3.838058e+06	3.838058e+06	3.838058e+06	3.838058e+06	0.000000e+00	neg
59905	4.526177e+06	4.515325e+06	4.515325e+06	4.515325e+06	4.529375e+06	neg
59906	5.944400e+04	5.944400e+04	5.944400e+04	5.944400e+04	0.000000e+00	neg
59907	2.272584e+06	2.272584e+06	2.272584e+06	2.272584e+06	0.000000e+00	neg
59908	4.486580e+06	4.486580e+06	4.486580e+06	4.486580e+06	0.000000e+00	neg
59909	3.580316e+06	3.580316e+06	3.580316e+06	3.580316e+06	0.000000e+00	neg
59910	1.413160e+05	1.413160e+05	1.413160e+05	1.413160e+05	0.000000e+00	neg
59911	4.240524e+07	4.240524e+07	4.240524e+07	4.240524e+07	4.529375e+06	neg
59912	1.898144e+06	1.898144e+06	1.898144e+06	1.898144e+06	0.000000e+00	neg
59913	2.457406e+06	2.457406e+06	2.457406e+06	2.457406e+06	0.000000e+00	neg
59914	2.501560e+06	2.501560e+06	2.501560e+06	2.501560e+06	0.000000e+00	neg
59915	9.929166e+06	9.929166e+06	9.929166e+06	9.929166e+06	4.529375e+06	neg
59916	3.231444e+06	3.231444e+06	3.231444e+06	3.231444e+06	0.000000e+00	neg
59917	2.921600e+04	2.921600e+04	2.921600e+04	2.921600e+04	0.000000e+00	neg
59918	7.990000e+04	7.990000e+04	7.990000e+04	7.990000e+04	0.000000e+00	neg
59919	7.921800e+04	7.921800e+04	7.921800e+04	7.921800e+04	0.000000e+00	neg
59920	4.862346e+06	4.862346e+06	4.862346e+06	4.862346e+06	0.000000e+00	neg
59921	5.554400e+04	5.554400e+04	5.554400e+04	5.554400e+04	0.000000e+00	neg
59922	3.497098e+06	3.497098e+06	3.497098e+06	3.497098e+06	0.000000e+00	neg
59923	1.180340e+05	1.180340e+05	1.180340e+05	1.180340e+05	0.000000e+00	neg
59924	5.292942e+06	5.292942e+06	5.292942e+06	5.292942e+06	0.000000e+00	neg
59925	3.660552e+06	3.660552e+06	3.660552e+06	3.660552e+06	0.000000e+00	neg
59926	3.603322e+06	3.603322e+06	3.603322e+06	3.603322e+06	0.000000e+00	neg
59927	3.201156e+06	3.201156e+06	3.201156e+06	3.201156e+06	0.000000e+00	neg
59928	2.905740e+06	2.905740e+06	2.905740e+06	2.905740e+06	0.000000e+00	neg
59929	5.289400e+04	5.289400e+04	5.289400e+04	5.289400e+04	1.500000e+03	neg
59930	2.493248e+06	2.493248e+06	2.493248e+06	2.493248e+06	0.000000e+00	neg
59931	2.877400e+04	4.515325e+06	4.515325e+06	4.515325e+06	3.838000e+03	neg
59932	3.861000e+04	3.861000e+04	3.861000e+04	3.861000e+04	0.000000e+00	neg
59933	5.070394e+06	5.070394e+06	5.070394e+06	5.070394e+06	0.000000e+00	neg
59934	3.550020e+06	3.550020e+06	3.550020e+06	3.550020e+06	0.000000e+00	neg
59935	6.784950e+06	6.784950e+06	6.784950e+06	6.784950e+06	0.000000e+00	neg
59936	3.017344e+06	3.017344e+06	3.017344e+06	3.017344e+06	0.000000e+00	neg
59937	2.671074e+06	2.671074e+06	2.671074e+06	2.671074e+06	0.000000e+00	neg
59938	4.714260e+05	4.714260e+05	4.714260e+05	4.714260e+05	0.000000e+00	neg
59939	1.920536e+07	1.920536e+07	1.920536e+07	1.920536e+07	0.000000e+00	neg
59940	4.991790e+06	4.991790e+06	4.991790e+06	4.991790e+06	0.000000e+00	neg
59941	1.152000e+04	1.152000e+04	1.152000e+04	1.152000e+04	0.000000e+00	neg
59942	6.694400e+04	6.694400e+04	6.694400e+04	6.694400e+04	0.000000e+00	neg
59943	2.392940e+06	2.392940e+06	2.392940e+06	2.392940e+06	0.000000e+00	neg
59944	8.756000e+04	8.756000e+04	8.756000e+04	8.756000e+04	0.000000e+00	neg
59945	1.815600e+04	1.815600e+04	1.815600e+04	1.815600e+04	2.000000e+01	neg
59946	3.464200e+04	3.464200e+04	3.464200e+04	3.464200e+04	1.808000e+03	neg
59947	3.397200e+04	3.397200e+04	3.397200e+04	3.397200e+04	5.960000e+03	neg
59948	2.496046e+06	2.496046e+06	2.496046e+06	2.496046e+06	0.000000e+00	neg
59949	3.652208e+06	3.652208e+06	3.652208e+06	3.652208e+06	0.000000e+00	neg
59950	1.924358e+07	1.924358e+07	1.924358e+07	1.924358e+07	0.000000e+00	neg
59951	1.068650e+07	1.068650e+07	1.068650e+07	1.068650e+07	0.000000e+00	neg
59952	8.306800e+04	8.306800e+04	8.306800e+04	8.306800e+04	0.000000e+00	neg
59953	2.652958e+06	2.652958e+06	2.652958e+06	2.652958e+06	0.000000e+00	neg
59954	2.911796e+06	2.911796e+06	2.911796e+06	2.911796e+06	0.000000e+00	neg
59955	2.535598e+06	2.535598e+06	2.535598e+06	2.535598e+06	0.000000e+00	neg
59956	2.658310e+06	2.658310e+06	2.658310e+06	2.658310e+06	0.000000e+00	neg
59957	6.651340e+05	6.651340e+05	6.651340e+05	6.651340e+05	0.000000e+00	neg
59958	2.566324e+06	2.566324e+06	2.566324e+06	2.566324e+06	0.000000e+00	neg
59959	5.484486e+06	5.484486e+06	5.484486e+06	5.484486e+06	0.000000e+00	neg
59960	3.207476e+06	3.207476e+06	3.207476e+06	3.207476e+06	0.000000e+00	neg
59961	3.346852e+06	3.346852e+06	3.346852e+06	3.346852e+06	0.000000e+00	neg
59962	1.081380e+05	1.081380e+05	1.081380e+05	1.081380e+05	0.000000e+00	neg
59963	1.097960e+05	1.097960e+05	1.097960e+05	1.097960e+05	0.000000e+00	neg
59964	3.140696e+06	3.140696e+06	3.140696e+06	3.140696e+06	0.000000e+00	neg
59965	1.578200e+04	1.578200e+04	1.578200e+04	1.578200e+04	7.760000e+02	neg
59966	1.451600e+04	1.451600e+04	1.451600e+04	1.451600e+04	1.280000e+02	neg
59967	2.019175e+07	2.019175e+07	2.019175e+07	2.019175e+07	4.529375e+06	neg
59968	2.579452e+06	2.579452e+06	2.579452e+06	2.579452e+06	0.000000e+00	neg
59969	5.415654e+06	5.415654e+06	5.415654e+06	5.415654e+06	0.000000e+00	neg
59970	2.759800e+04	2.759800e+04	2.759800e+04	2.759800e+04	8.660000e+02	neg
59971	2.937708e+06	2.937708e+06	2.937708e+06	2.937708e+06	0.000000e+00	neg
59972	1.614600e+04	1.614600e+04	1.614600e+04	1.614600e+04	1.322000e+03	neg
59973	4.658400e+05	4.658400e+05	4.658400e+05	4.658400e+05	0.000000e+00	neg
59974	5.477608e+06	5.477608e+06	5.477608e+06	5.477608e+06	0.000000e+00	neg
59975	2.652902e+06	2.652902e+06	2.652902e+06	2.652902e+06	0.000000e+00	neg
59976	1.437086e+06	1.437086e+06	1.437086e+06	1.437086e+06	0.000000e+00	neg
59977	1.938484e+06	1.938484e+06	1.938484e+06	1.938484e+06	2.452470e+06	neg
59978	4.368640e+06	4.368640e+06	4.368640e+06	4.368640e+06	0.000000e+00	neg
59979	2.636600e+04	2.636600e+04	2.636600e+04	2.636600e+04	1.620000e+02	neg
59980	7.299000e+04	7.299000e+04	7.299000e+04	7.299000e+04	2.342600e+04	neg
59981	1.573620e+05	1.573620e+05	1.573620e+05	1.573620e+05	0.000000e+00	neg
59982	8.509880e+05	8.509880e+05	8.509880e+05	8.509880e+05	0.000000e+00	neg
59983	3.076786e+06	3.076786e+06	3.076786e+06	3.076786e+06	0.000000e+00	neg
59984	2.567934e+06	2.567934e+06	2.567934e+06	2.567934e+06	0.000000e+00	neg
59985	4.792826e+06	4.792826e+06	4.792826e+06	4.792826e+06	0.000000e+00	neg
59986	2.101800e+05	2.101800e+05	2.101800e+05	2.101800e+05	1.663000e+04	neg
59987	4.131930e+06	4.131930e+06	4.131930e+06	4.131930e+06	0.000000e+00	neg
59988	3.591740e+05	3.591740e+05	3.591740e+05	3.591740e+05	0.000000e+00	neg
59989	4.296150e+06	4.296150e+06	4.296150e+06	4.296150e+06	0.000000e+00	neg
59990	5.272338e+06	5.272338e+06	5.272338e+06	5.272338e+06	0.000000e+00	neg
59991	2.506358e+06	2.506358e+06	2.506358e+06	2.506358e+06	0.000000e+00	neg
59992	1.658400e+04	1.658400e+04	1.658400e+04	1.658400e+04	0.000000e+00	neg
59993	1.329000e+04	1.329000e+04	1.329000e+04	1.329000e+04	0.000000e+00	neg
59994	5.720200e+04	5.720200e+04	5.720200e+04	5.720200e+04	0.000000e+00	neg
59995	1.084523e+07	1.084523e+07	1.084523e+07	1.084523e+07	0.000000e+00	neg
59996	1.476380e+05	1.476380e+05	1.476380e+05	1.476380e+05	0.000000e+00	neg
59997	4.373800e+04	4.373800e+04	4.373800e+04	4.373800e+04	4.544000e+03	neg
59998	6.159728e+06	6.159728e+06	6.159728e+06	6.159728e+06	0.000000e+00	neg
59999	3.076406e+06	3.076406e+06	3.076406e+06	3.076406e+06	0.000000e+00	neg
60000 rows × 6 columns

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-37-d9b9564a3755> in <module>()
     24     classifier = DecisionTreeClassifier(class_weight={'pos':.983, 'neg':.017},max_depth =3)
     25     classifier.fit(pd.DataFrame(data=featureSelectedMeanTrainDf,columns=columnList), pd.DataFrame(data=featureSelectedMeanTrainDf,columns=['class']))
---> 26     predictions = classifier.predict(pd.DataFrame(data=featureSelectedMeanTestDf,columns=columnList))
     27     score = evaluate(pd.DataFrame(data=featureSelectedMeanTestDf,columns=['class']), predictions)
     28 

~\Anaconda3\envs\ATIML\lib\site-packages\sklearn\tree\tree.py in predict(self, X, check_input)
    410         """
    411         check_is_fitted(self, 'tree_')
--> 412         X = self._validate_X_predict(X, check_input)
    413         proba = self.tree_.predict(X)
    414         n_samples = X.shape[0]

~\Anaconda3\envs\ATIML\lib\site-packages\sklearn\tree\tree.py in _validate_X_predict(self, X, check_input)
    371         """Validate X whenever one tries to predict, apply, predict_proba"""
    372         if check_input:
--> 373             X = check_array(X, dtype=DTYPE, accept_sparse="csr")
    374             if issparse(X) and (X.indices.dtype != np.intc or
    375                                 X.indptr.dtype != np.intc):

~\Anaconda3\envs\ATIML\lib\site-packages\sklearn\utils\validation.py in check_array(array, accept_sparse, dtype, order, copy, force_all_finite, ensure_2d, allow_nd, ensure_min_samples, ensure_min_features, warn_on_dtype, estimator)
    451                              % (array.ndim, estimator_name))
    452         if force_all_finite:
--> 453             _assert_all_finite(array)
    454 
    455     shape_repr = _shape_repr(array.shape)

~\Anaconda3\envs\ATIML\lib\site-packages\sklearn\utils\validation.py in _assert_all_finite(X)
     42             and not np.isfinite(X).all()):
     43         raise ValueError("Input contains NaN, infinity"
---> 44                          " or a value too large for %r." % X.dtype)
     45 
     46 

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').
In [7]:
#Implementation of scikit Support vector Machine(Linear)
#################################################################################
linearSvm = svm.LinearSVC(class_weight={'pos':.983, 'neg':.017})

target = pd.DataFrame(data=mostFrequentTrainSet,columns=['class'])

linearSvm.fit(pd.DataFrame(data=mostFrequentTrainSet,columns=COLUMNS),target.values.ravel())  
predictions = linearSvm.predict(pd.DataFrame(data=mostFrequentTestSet,columns=COLUMNS))  
score = evaluate(pd.DataFrame(data=mostFrequentTestSet,columns=['class']), predictions)
             precision    recall  f1-score   support

        neg       0.99      0.99      0.99     15625
        pos       0.65      0.70      0.67       375

avg / total       0.98      0.98      0.98     16000

Confusion matrix, without normalization
[[15484   141]
 [  114   261]]
Normalized confusion matrix
[[0.990976 0.009024]
 [0.304    0.696   ]]


Score: 58410